In [2]:
from sklearn.datasets import make_multilabel_classification
from sklearn.preprocessing import MultiLabelBinarizer

In [87]:
X, Y = make_multilabel_classification(n_samples=3, random_state=0,
                                      return_indicator=False)

In [88]:
Y

[[2, 3, 4], [2], [0, 1, 3]]

In [89]:
MultiLabelBinarizer().fit_transform(Y)

array([[0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0],
       [1, 1, 0, 1, 0]])

In [204]:
import numpy as np
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X, y = iris.data, iris.target
predarray = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)
print(f'X:{X}')
print(f'y:{y}')
print(numpy.corrcoef(y,predarray))

X:[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 

/Users/tylergroshong/opt/miniconda3/envs/ml1/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/tylergroshong/opt/miniconda3/envs/ml1/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [258]:
import warnings
warnings.filterwarnings("ignore")

X, y = iris.data, iris.target
y = list(y)
for i in range(len(y)):
    if (y[i] == 0):
        y[i] = ("Iris-Setosa,",)
    elif (y[i] == 1):
        y[i] = ("Iris-Versicolour,",)
    elif (y[i] == 2):
        y[i] = ("Iris-Virginica,",)

ymlb = MultiLabelBinarizer()
yfit = ymlb.fit_transform(y)

def train_on_n_samples(n):
    print(f'hello: {n}')
    predarray = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X[:n], yfit[:n]).predict(X)
    sum = 0
    for i in range(len(yfit)):
        if (np.array_equal(yfit[i],predarray[i])):
            sum+=1
    return(sum/len(y))

for j in range(1,len(y)):
    print(train_on_n_samples(j))

hello: 1
0.3333333333333333
hello: 2
0.3333333333333333
hello: 3
0.3333333333333333
hello: 4
0.3333333333333333
hello: 5
0.3333333333333333
hello: 6
0.3333333333333333
hello: 7
0.3333333333333333
hello: 8
0.3333333333333333
hello: 9
0.3333333333333333
hello: 10
0.3333333333333333
hello: 11
0.3333333333333333
hello: 12
0.3333333333333333
hello: 13
0.3333333333333333
hello: 14
0.3333333333333333
hello: 15
0.3333333333333333
hello: 16
0.3333333333333333
hello: 17
0.3333333333333333
hello: 18
0.3333333333333333
hello: 19
0.3333333333333333
hello: 20
0.3333333333333333
hello: 21
0.3333333333333333
hello: 22
0.3333333333333333
hello: 23
0.3333333333333333
hello: 24
0.3333333333333333
hello: 25
0.3333333333333333
hello: 26
0.3333333333333333
hello: 27
0.3333333333333333
hello: 28
0.3333333333333333
hello: 29
0.3333333333333333
hello: 30
0.3333333333333333
hello: 31
0.3333333333333333
hello: 32
0.3333333333333333
hello: 33
0.3333333333333333
hello: 34
0.3333333333333333
hello: 35
0.33333333333